In [3]:
pip install xgboost


   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/72.0 MB 544.7 kB/s eta 0:02:13
   ---------------------------------------- 0.1/72.0 MB 655.4 kB/s eta 0:01:50
   ---------------------------------------- 0.1/72.0 MB 490.2 kB/s eta 0:02:27
   ---------------------------------------- 0.2/72.0 MB 655.4 kB/s eta 0:01:50
   ---------------------------------------- 0.2/72.0 MB 808.4 kB/s eta 0:01:29
   ---------------------------------------- 0.3/72.0 MB 714.4 kB/s eta 0:01:41
   ---------------------------------------- 0.5/72.0 MB 1.1 MB/s eta 0:01:04
   ---------------------------------------- 0.5/72.0 MB 1.2 MB/s eta 0:01:02
   ---------------------------------------- 0.5/72.0 MB 1.0 MB/s eta 0:01:10
   ---------------------------------------- 0.7/72.0 MB 1.3 MB/s eta 0:00:58
   ---------------------------------------- 0.7/72.0 MB 1.3 MB/s eta 0:00:58
 

In [40]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 435.7 kB/s eta 1:33:42
     ---------------------------------------- 0.0/2.4 GB 871.5 kB/s eta 0:46:51
     ---------------------------------------- 0.0/2.4 GB 1.2 MB/s eta 0:35:08
     ---------------------------------------- 0.0/2.4 GB 1.7 MB/s eta 0:23:39
     ---------------------------------------- 0.0/2.4 GB 2.1 MB/s eta 0:19:51
     ---------------------------------------- 0.0/2.4 GB 2.4 MB/s eta 0:16:43
     ---------------------------------------- 0.0/2.4 GB 3.1 MB/s eta 0:13:21
     ---------------------------------------- 0.0/2.4 GB 3.6 MB/s eta 0:11:22
     ---------------------------------------- 0.0/2.4 GB 3.8 MB/s eta 0:10:41
     ---------------------------------------- 0.0/2.4 GB 3.8 MB/s eta 0:10:41
     ---------------------------------------- 0.0/2.4 GB 3.8 MB/s eta 0:10:41
     ---

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Legion-User\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Legion-User\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Legion-User\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Legion-User\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Legion-User\anaconda3\Lib\http\client.py", line 473, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\Legion-User\anaconda3\Lib\socket.py", line 706, in readinto
    return 

In [39]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'torch'

In [4]:
# Load data
X_train = pd.read_csv("X_train_scaled.csv")
X_test = pd.read_csv("X_test_scaled.csv")
y_train = pd.read_csv("y_train.csv")
y_test = pd.read_csv("y_test.csv")

In [16]:
# ============================================
# 1️⃣ LOAD DATA
# ============================================

df = pd.read_csv("housepricefeature.csv")  

print("✅ Data Loaded Successfully!")
print(df.head(), "\n")
print(df.info())

# ============================================
# 3️⃣ FEATURE ENGINEERING
# ============================================
# Create new derived numerical features safely
# (Convert to numeric if necessary to avoid string issues)
numeric_cols = ['Median_Price', 'Area', 'Transactions', 'Median_PSF']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Derived ratio features
df["Price_per_Sqft"] = df["Median_Price"] / (df["Area"] + 1)
df["Transactions_per_Area"] = df["Transactions"] / (df["Area"] + 1)
df["Value_Index"] = df["Median_Price"] / (df["Median_PSF"] + 1)

# Replace missing values
df = df.fillna(0)

# ============================================
# 4️⃣ FEATURE & TARGET SELECTION
# ============================================
target_col = "Median_Price"   # Change if your target column differs
X = df.drop(columns=[target_col])
y = df[target_col]

# Optional: Apply log transform to target (stabilize variance)
y = np.log1p(y)

# ============================================
# 5️⃣ ENCODE CATEGORICAL VARIABLES
# ============================================
print("🔍 Checking for categorical columns...")
cat_cols = X.select_dtypes(include=['object']).columns
print(f"Found categorical columns: {list(cat_cols)}")

# Encode categorical variables
X = pd.get_dummies(X, drop_first=True)

# 🧹 Clean column names (important for XGBoost)
X.columns = X.columns.str.replace('[\[\]<]', '', regex=True)

# Fill missing values
X = X.fillna(0)

print("✅ All categorical variables encoded and cleaned.")
print(f"Total features after encoding: {X.shape[1]}")
# ============================================
# 6️⃣ TRAIN / TEST SPLIT
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"✅ Training samples: {X_train.shape[0]}")
print(f"✅ Testing samples : {X_test.shape[0]}")

# ============================================
# 7️⃣ TRAIN XGBOOST MODEL
# ============================================
xgb = XGBRegressor(
    n_estimators=1000,        # number of boosting trees
    learning_rate=0.05,       # smaller = more accurate (but slower)
    max_depth=8,              # controls model complexity
    subsample=0.8,            # prevents overfitting
    colsample_bytree=0.8,     # column sampling
    random_state=42,
    objective="reg:squarederror"
)

print("🚀 Training XGBoost model...")
xgb.fit(X_train, y_train)

# ============================================
# 8️⃣ PREDICT & EVALUATE
# ============================================
y_pred_log = xgb.predict(X_test)
y_pred = np.expm1(y_pred_log)   # reverse log transform
y_test_actual = np.expm1(y_test)

mae = mean_absolute_error(y_test_actual, y_pred)
rmse = np.sqrt(mean_squared_error(y_test_actual, y_pred))
r2 = r2_score(y_test_actual, y_pred)

print("\n🎯 XGBoost Regression Results")
print("-------------------------------------")
print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")
print(f"R²   : {r2:.4f}")

✅ Data Loaded Successfully!
                      Township           Area   State     Tenure  \
0          SCIENTEX SUNGAI DUA  Tasek Gelugor  Penang   Freehold   
1                 BANDAR PUTRA          Kulai   Johor   Freehold   
2  TAMAN LAGENDA TROPIKA TAPAH    Chenderiang   Perak   Freehold   
3       SCIENTEX JASIN MUTIARA         Bemban  Melaka   Freehold   
4           TAMAN LAGENDA AMAN          Tapah   Perak  Leasehold   

                           Type  Median_Price  Median_PSF  Transactions  \
0                 Terrace House      331800.0       304.0           593   
1  Cluster House, Terrace House      590900.0       322.0           519   
2                 Terrace House      229954.0       130.0           414   
3                 Terrace House      255600.0       218.0           391   
4                 Terrace House      219300.0       168.0           363   

   Price_per_Transaction  HighDemand  ... Is_Central  Log_Median_Price  \
0             558.585859           1  

In [24]:
# ============================================
# 1️⃣ LOAD DATA
# ============================================
df = pd.read_csv("housepricefeature.csv")  # your dataset file
print("✅ Data loaded:", df.shape)

# Identify columns
num_cols = ["Area", "Median_Price", "Median_PSF", "Transactions"]
cat_cols = ["Township", "State", "Tenure", "Type"]

# Convert numeric columns
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing numeric with median
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill missing categorical with 'Unknown'
for col in cat_cols:
    df[col] = df[col].fillna("Unknown")

# ============================================
# 2️⃣ FEATURE ENGINEERING
# ============================================
df["Price_per_Sqft"] = df["Median_Price"] / (df["Area"] + 1)
df["Transactions_per_Area"] = df["Transactions"] / (df["Area"] + 1)
df["Value_Index"] = df["Median_Price"] / (df["Median_PSF"] + 1)

# Add derived categorical interactions (if exist in your dataset)
# Example: these might already be added from earlier preprocessing
if "PriceTier" not in df.columns:
    df["PriceTier"] = pd.qcut(df["Median_Price"], q=3, labels=["Low", "Medium", "High"])

if "Transaction_Volume_Flag" not in df.columns:
    df["Transaction_Volume_Flag"] = np.where(df["Transactions"] > df["Transactions"].median(), "High", "Low")

# ============================================
# 3️⃣ ENCODING ALL CATEGORICALS
# ============================================
cat_cols_full = [col for col in df.columns if df[col].dtype == "object" or df[col].dtype.name == "category"]

df_encoded = pd.get_dummies(df, columns=cat_cols_full, drop_first=True)
df_encoded.columns = df_encoded.columns.str.replace(r"[\[\]<]", "", regex=True)

# ============================================
# 4️⃣ FEATURES & TARGET
# ============================================
X = df_encoded.drop(columns=["Median_Price"])
y = df_encoded["Median_Price"]

# Scale only numeric columns
scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)

# ============================================
# 5️⃣ SPLIT DATA
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"✅ Train: {X_train.shape}, Test: {X_test.shape}")

# ============================================
# 6️⃣ XGBOOST AUTO-TUNING
# ============================================
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)

param_grid = {
    "n_estimators": [300, 500, 800],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [4, 6, 8],
    "subsample": [0.7, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.9, 1.0],
    "min_child_weight": [1, 3, 5],
    "gamma": [0, 0.1, 0.3]
}

print("🔍 Running auto-tuning...")
xgb_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=20,
    scoring="r2",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

xgb_search.fit(X_train, y_train)

# ============================================
# 7️⃣ EVALUATE BEST MODEL
# ============================================
best_xgb = xgb_search.best_estimator_
y_pred = best_xgb.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n🎯 XGBoost Auto-Tuned Results")
print("-------------------------------------")
print(f"Best Params : {xgb_search.best_params_}")
print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")
print(f"R²   : {r2:.4f}")



✅ Data loaded: (2000, 22)


C:\Users\Legion-User\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Legion-User\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Legion-User\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


✅ Train: (1600, 2116), Test: (400, 2116)
🔍 Running auto-tuning...
Fitting 3 folds for each of 20 candidates, totalling 60 fits

🎯 XGBoost Auto-Tuned Results
-------------------------------------
Best Params : {'subsample': 1.0, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 1.0}
MAE  : 5,978.96
RMSE : 38,731.12
R²   : 0.9915
